In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
import seaborn as sns
df = pd.read_csv("/content/drive/My Drive/Dataset/data-mining/weatherHistory.csv")

In [29]:
# Preprocessing
df = df.dropna()
andIdx = df[df['Summary'].str.contains("and")].index
df.drop(andIdx , inplace=True)
label_encoder = preprocessing.LabelEncoder()
df['Precip Type'] = label_encoder.fit_transform(df['Precip Type'])
df['Summary'] = label_encoder.fit_transform(df['Summary'])
med = df['Precip Type'].median()
df['Precip Type'].fillna(med, inplace=True)

In [30]:
# Prepare Data Train & Data Test
features = ['Precip Type', 'Temperature (C)', 'Apparent Temperature (C)',
            'Humidity', 'Wind Speed (km/h)', 'Wind Bearing (degrees)',
            'Visibility (km)', 'Pressure (millibars)']
target = 'Summary'

x_train, x_test, y_train, y_test = train_test_split(df[features], df[target],
                                                    train_size=0.70, test_size=0.30, shuffle=False)

# print("X_train: {}, Y_train: {}".format(len(x_train), len(x_test)))
# print("X_train: {}, Y_train: {}".format(len(y_train), len(y_test)))

In [31]:
# Train Model Bagging
# base : Decision Tree Classifier
dtc = DecisionTreeClassifier(criterion="entropy")
bag_model = BaggingClassifier(base_estimator=dtc, n_estimators=100, bootstrap=True)
bag_model = bag_model.fit(x_train, y_train)

In [ ]:
# Lama di run
# Train Model Bagging
# base : Logistic Regression
lr = LogisticRegression()
bag_model = BaggingClassifier(base_estimator=lr, n_estimators=100, bootstrap=True)
bag_model = bag_model.fit(x_train, y_train)

In [20]:
# Train Model Bagging
# base : Bernoulli Naive Bayes
bnb = BernoulliNB()
bag_model = BaggingClassifier(base_estimator=bnb, n_estimators=100, bootstrap=True)
bag_model = bag_model.fit(x_train, y_train)

In [ ]:
# Lama di run
# Train Model Bagging
# base : Gaussian Naive Bayes
gnb = GaussianNB()
bag_model = BaggingClassifier(base_estimator=gnb, n_estimators=100, bootstrap=True)
bag_model = bag_model.fit(x_train, y_train)

In [36]:
predicted = bag_model.predict(x_test)
print(bag_model.score(x_test, y_test))

0.4626849819443461


In [37]:
print(confusion_matrix(y_test, predicted))

[[  12    2    0    0    0    0    0    6    2    0    0]
 [   1  751    0    0    1    0  424  265  813    0    0]
 [   0    5    0    0    8    0   11    4    6    0    0]
 [   0    0    0    0    0    0    0    0    1    0    0]
 [   0    3    0    0 1908    0    0    0    0    0    0]
 [   0    7    0    0    5    0   10   18    6    0    0]
 [   3  874    0    1    0    0 3911 1145 3809    0    0]
 [   1  292    0    0    0    0 2117 1900 1147    0    0]
 [   5 1519    0    0    0    0 2183  472 4584    0    0]
 [   0    1    0    0    6    0    0    3    0    0    0]
 [   1    0    0    0    0    0    0    0    0    0    3]]


In [38]:
# Measure Performance
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.52      0.55      0.53        22
           1       0.22      0.33      0.26      2255
           2       0.00      0.00      0.00        34
           3       0.00      0.00      0.00         1
           4       0.99      1.00      0.99      1911
           5       0.00      0.00      0.00        46
           6       0.45      0.40      0.43      9743
           7       0.50      0.35      0.41      5457
           8       0.44      0.52      0.48      8763
           9       0.00      0.00      0.00        10
          10       1.00      0.75      0.86         4

    accuracy                           0.46     28246
   macro avg       0.37      0.35      0.36     28246
weighted avg       0.47      0.46      0.46     28246



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
# Classification Result
precipType = 0
temperature = 9.472222
apparentTemp = 7.388889
humidity = 0.89
windSpeed = 14.1197
windBearing = 251
visibility = 15.8263
pressure = 1015.13
predicted = bag_model.predict([[precipType, temperature,
                            apparentTemp, humidity,
                            windSpeed, windBearing,
                            visibility, pressure]])
result = ""

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  "X does not have valid feature names, but"


In [40]:
if predicted[0] == 0:
  result = "Breezy"
elif predicted[0] == 1:
  result = "Clear"
elif predicted[0] == 2:
  result = "Drizzle"
elif predicted[0] == 3:
  result = "Dry"
elif predicted[0] == 4:
  result = "Foggy"
elif predicted[0] == 5:
  result = "Light Rain"
elif predicted[0] == 6:
  result = "Mostly Cloudy"
elif predicted[0] == 7:
  result = "Overcast"
elif predicted[0] == 8:
  result = "Partly Cloudy"
elif predicted[0] == 9:
  result = "Rain"
elif predicted[0] == 10:
  result = "Windy"
else:
  result = "Not Found"
print("Prediction Result :", result)

Prediction Result : Partly Cloudy
